In [49]:
import numpy as np
import random
from tqdm.auto import tqdm

In [50]:
#Start the game
board = [' '] * 9
player_symbol = 'X'
opponent_symbol = 'O'

In [51]:
def get_state():
    return tuple(board)

In [52]:
Q_table = {}

learning_rate = 0.4 #Sarebbe il parametro ALPHA--> Ho messo un valore arbitrario
discount_factor = 0.7 # Sarebbe il termine PHI visto a lezione nelle slide
exploration_prob = 0.3 #Ci serve per creare lo stato del nostro utente a seguito di un'azione che faccio
num_episodes = 500_000 #Numero di cicli che uso

# Funzione per inizializzare la tabella Q per uno stato specifico
def initialize_Q(state):
    if state not in Q_table:
        Q_table[state] = {i: 0 for i in range(9)}

def print_board():
    print(f" {board[0]} | {board[1]} | {board[2]} ")
    print("---|---|---")
    print(f" {board[3]} | {board[4]} | {board[5]} ")
    print("---|---|---")
    print(f" {board[6]} | {board[7]} | {board[8]} ")

In [53]:
# Funzione per eseguire una mossa
def make_move(position, symbol):
    board[position] = symbol

# Funzione per valutare lo stato del gioco
def evaluate_state():
    for i in range(0, 9, 3):
        if board[i] == board[i + 1] == board[i + 2] != ' ':
            return 1 if board[i] == player_symbol else -1
    for i in range(3):
        if board[i] == board[i + 3] == board[i + 6] != ' ':
            return 1 if board[i] == player_symbol else -1
    if board[0] == board[4] == board[8] != ' ':
        return 1 if board[0] == player_symbol else -1
    if board[2] == board[4] == board[6] != ' ':
        return 1 if board[2] == player_symbol else -1
    if ' ' not in board:
        return 0
    return None

In [54]:
# Parametri dell'algoritmo Q-learning
initial_exploration_prob = 1.0
final_exploration_prob = 0.5
exploration_decay = num_episodes //1

# Funzione per selezionare l'azione in base alla politica di esplorazione/esercitazione per il giocatore
def select_player_action(state, episode):

    exploration_prob = final_exploration_prob + (initial_exploration_prob - final_exploration_prob) * np.exp(-episode / exploration_decay)

    if random.uniform(0, 1) < exploration_prob:
        return random.choice([i for i in range(9) if board[i] == ' '])

    else:
        return max(Q_table[state], key=Q_table[state].get)

# Funzione per selezionare l'azione dell'avversario in modo casuale (esplorazione casuale)
def select_opponent_action():
    return random.choice([i for i in range(9) if board[i] == ' '])



In [55]:

wins_X = 0
wins_O = 0
deuce=0
# Algoritmo Q-learning con giocatore che inizia con valore della X prima
for episode in tqdm(range(num_episodes)):
    # Reset del gioco
    board = [' '] * 9
    reward = 0
    while True:
        # Stato corrente
        state = get_state()
        # Selezione dell'azione per il giocatore
        action = select_player_action(state,episode)
        # Esegui la mossa del giocatore
        make_move(action, player_symbol)
        # Valuta lo stato dopo la mossa del giocatore
        result = evaluate_state()

        if result is not None:
            reward = result
            if reward == 1:
              wins_X += 1
            elif reward == -1:
              wins_O += 1
            else:
              deuce+=1
            break  # Il gioco è terminato
        # Stato successivo
        next_state = get_state()
        # Selezione dell'azione dell'avversario (casuale)
        opponent_action = select_opponent_action()
        # Esegui la mossa dell'avversario
        make_move(opponent_action, opponent_symbol)
        # Valuta lo stato dopo la mossa dell'avversario
        result = evaluate_state()

        if result is not None:
            reward = result
            if reward == 1:
             wins_X += 1
            elif reward == -1:
             wins_O += 1
            else:
              deuce+=1
            break  # Il gioco è terminato

        # Stato successivo
        new_state = get_state()

        # Aggiornamento della tabella Q solo per il giocatore
        initialize_Q(state)
        initialize_Q(new_state)
        Q_table[state][action] = (1 - learning_rate) * Q_table[state][action] + learning_rate * (reward + discount_factor * max(Q_table[new_state].values()) - Q_table[state][action])
total_games = num_episodes
win_percentage_X = (wins_X / total_games) * 100
win_percentage_O = (wins_O / total_games) * 100

  0%|          | 0/500000 [00:00<?, ?it/s]

In [56]:
print(f"\nLa lunghezza della tabella Q che uso per l'esplorazione è:{len(Q_table)}\n")
print(f"Percentuale di vittorie per X: {win_percentage_X}%")
print(f"Percentuale di vittorie per O: {win_percentage_O}%")
print_board()



La lunghezza della tabella Q che uso per l'esplorazione è:4041

Percentuale di vittorie per X: 50.2638%
Percentuale di vittorie per O: 37.5678%
 X |   | O 
---|---|---
 X |   |   
---|---|---
 X | O |   
